In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/")

%cd "/content/gdrive/My Drive/Colab Notebooks/bikes-theft-model"
!pwd

# Libraries: Standard ones
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Library for boxplots
import seaborn as sns
import pandas as pd

#GRAPHS CLASS
from Codigos.DataStatistics import GraphsStatistics as gp

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
/content/gdrive/My Drive/Colab Notebooks/bikes-theft-model
/content/gdrive/My Drive/Colab Notebooks/bikes-theft-model


In [ ]:
data = pd.read_csv('Data/Bicycle_Theft_Ottawa.csv', index_col= 0)
display(data.sample(6))


,Division,Sector,Report_Date,Incident_Date,Incident_Weekday,Location_Type,Primary_Offence,Neighbourhood,Bicycle_Status,Bicycle_Value,Bicycle_Make,Bicycle_Model,Bicycle_Type,Bicycle_Frame,Bicycle_Colour,Bicycle_Speed,x,y,ObjectId
ID,,,,,,,,,,,,,,,,,,,
1407,West,16,8/17/16 9:01 PM,8/17/16 6:00 PM,4,PARKING LOTS,THEFT<BICYCLE,Chapman Mills,STOLEN,200.0,SIMS,BMX,BMX,CHILD'S,Red,0,-75.719367,45.273658,1107
747,Central,23,7/14/18 9:06 AM,7/13/18 11:00 AM,6,OTHER NON COMMERCIAL/CORPORATE PLACES/ GOVT BLDG,THEFT<BICYCLE,Centretown,STOLEN,1000.0,CANNONDALE,NaN,RACER,MEN'S,White,18,-75.701208,45.417623,947
1501,Central,25,8/24/17 1:58 PM,8/23/17 10:30 AM,4,"STREETS, ROADS, HIGHWAYS",THEFT<BICYCLE,Vanier South,STOLEN,600.0,SPORTEK,NaN,MOUNTAIN,MEN'S,Blue,21,-75.656628,45.433986,1302
4441,West,13,08/12/2019 22:19,08/12/2019 13:05,2,PARKING LOTS,THEFT<BICYCLE,Kanata Lakes - Arcardia,STOLEN,800.0,G.T.,AVALANCH 3.0,MOUNTAIN,MEN'S,Blue,26,-75.911636,45.310499,4592
581,Central,23,12/06/2017 11:48,12/05/2017 07:45,3,OPEN AREAS,THEFT<BICYCLE,Centretown,STOLEN,450.0,SCHWINN,BIGGITTY DLX,OTHER,MEN'S,Black,7,-75.694060,45.418968,895
174,Central,23,9/19/15 2:09 PM,9/18/15 10:30 PM,6,DWELLING UNIT,THEFT<BICYCLE,Centretown,STOLEN,650.0,KHS,FLITE 223,RACER,WOMEN'S,Red,0,-75.689464,45.410594,788


In [ ]:
data.shape

(5652, 19)

In [ ]:
data.describe()
gp.CountBar_plot

<function Codigos.DataStatistics.GraphsStatistics.CountBar_plot>

In [ ]:
data.describe(include='O')

,Division,Report_Date,Incident_Date,Location_Type,Primary_Offence,Neighbourhood,Bicycle_Status,Bicycle_Make,Bicycle_Model,Bicycle_Type,Bicycle_Frame,Bicycle_Colour
count,5652,5652,5652,5652,5652,5652,5652,4890,3367,5381,5337,5262
unique,3,5637,5366,29,30,105,5,574,2144,9,4,153
top,Central,6/24/16 3:19 PM,6/27/18 6:00 PM,"SINGLE HOME, HOUSE",THEFT<BICYCLE,Centretown,STOLEN,GIANT,ESCAPE,MOUNTAIN,MEN'S,Black
freq,3860,2,4,1173,5342,1127,5149,540,28,1884,3304,1273


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5652 entries, 1 to 5218
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Division          5652 non-null   object 
 1   Sector            5652 non-null   int64  
 2   Report_Date       5652 non-null   object 
 3   Incident_Date     5652 non-null   object 
 4   Incident_Weekday  5652 non-null   int64  
 5   Location_Type     5652 non-null   object 
 6   Primary_Offence   5652 non-null   object 
 7   Neighbourhood     5652 non-null   object 
 8   Bicycle_Status    5652 non-null   object 
 9   Bicycle_Value     5019 non-null   float64
 10  Bicycle_Make      4890 non-null   object 
 11  Bicycle_Model     3367 non-null   object 
 12  Bicycle_Type      5381 non-null   object 
 13  Bicycle_Frame     5337 non-null   object 
 14  Bicycle_Colour    5262 non-null   object 
 15  Bicycle_Speed     5652 non-null   int64  
 16  x                 5652 non-null   float64


In [ ]:
data.convert_dtypes().dtypes

Division             string
Sector                Int64
Report_Date          string
Incident_Date        string
Incident_Weekday      Int64
Location_Type        string
Primary_Offence      string
Neighbourhood        string
Bicycle_Status       string
Bicycle_Value         Int64
Bicycle_Make         string
Bicycle_Model        string
Bicycle_Type         string
Bicycle_Frame        string
Bicycle_Colour       string
Bicycle_Speed         Int64
x                   float64
y                   float64
ObjectId              Int64
dtype: object

In [ ]:
data[['Neighbourhood', 'Bicycle_Status']].head(10) #if 'Bicycle_Status' == 'STOLEN'

,Neighbourhood,Bicycle_Status
ID,,
1,Old Barrhaven West,STOLEN
2,Old Barrhaven West,STOLEN
3,Old Barrhaven West,STOLEN
4,Old Barrhaven West,RECOVERED
5,Old Barrhaven West,STOLEN
6,Old Barrhaven West,STOLEN
7,Old Barrhaven West,STOLEN
8,Old Barrhaven West,STOLEN
9,Old Barrhaven West,RECOVERED


In [ ]:
data[data['Bicycle_Status'] == 'STOLEN'].groupby("Neighbourhood").size().idxmax()

'Centretown'

In [ ]:
data.columns

Index(['Division', 'Sector', 'Report_Date', 'Incident_Date',
       'Incident_Weekday', 'Location_Type', 'Primary_Offence', 'Neighbourhood',
       'Bicycle_Status', 'Bicycle_Value', 'Bicycle_Make', 'Bicycle_Model',
       'Bicycle_Type', 'Bicycle_Frame', 'Bicycle_Colour', 'Bicycle_Speed', 'x',
       'y', 'ObjectId'],
      dtype='object')

In [ ]:
import plotly.express as px
 #Y = data.loc[data['Bicycle_Status'] == 'STOLEN'].groupby("Incident_Weekday").size()
 # data.groupby("Neighbourhood").size()
sub_Da = data[data['Bicycle_Status'] == 'STOLEN'].groupby("Incident_Weekday").size().reset_index().replace({1: 'Monday',2: 'Tuesday', 3: 'Weekday', 4: 'Thursday', 5: 'Friday', 6:'Saturday', 7:'Sunday'})
sub_Da.rename(columns={0:'Stolen Bicycles'}, inplace=True)

fig = px.bar(sub_Da, "Incident_Weekday", 'Stolen Bicycles', title='Stolen Bicycles')
fig.show()

In [16]:
data['Report_Date']=pd.to_datetime(data['Report_Date']) #Transforma la columna Formato de fecha
data['Week_day']=data['Report_Date'].dt.day_name() #Agregar el nombre
data['Year'] = data['Report_Date'].dt.year



In [19]:
Dias_aux_robo_year=data.groupby(['Year','Bicycle_Status']).size().reset_index().rename(columns={0:'Count'})
display(Dias_aux_robo_year)

fig = px.line(Dias_aux_robo_year, x='Year', y='Count', color='Bicycle_Status')
fig.show()

,Year,Bicycle_Status,Count
0,2015,FOUND,6
1,2015,RECOVERED,45
2,2015,SEIZED,4
3,2015,STOLEN,1018
4,2015,UNKNOWN,41
5,2016,FOUND,1
6,2016,RECOVERED,54
7,2016,SEIZED,3
8,2016,STOLEN,1020
9,2016,UNKNOWN,44
